In [30]:
import requests
import json

# Replace with the actual URL where the Ollama server is running
url = "http://192.168.1.208:11434/api/generate"  # Adjust the port if necessary

# Define the input data for the llama3 model
data = {
    "model": "llama-3-Korean-Bllossom-8B:latest",
    "prompt": "Why is the sky blue?"
}

# Set the headers, if necessary
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response status and print the output
if response.status_code == 200:
    result = response.json()
    print("Response from llama3 model:", result)
else:
    print("Failed to get response:", response.status_code, response.text)


Failed to get response: 404 {"error":"model 'llama-3-Korean-Bllossom-8B:latest' not found, try pulling it first"}


In [27]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch

# T5 모델 로드
# model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
# tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

# model = AutoModelForSeq2SeqLM.from_pretrained("theSOL1/kogrammar-distil")
# tokenizer = AutoTokenizer.from_pretrained("theSOL1/kogrammar-distil")

tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")
model = AutoModelForCausalLM.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "mps:0" if torch.cuda.is_available() else "cpu" # for mac m1

model = model.to(device)

# 예시 입력 문장
input_text = """
계약관리
규정에 따르면, 물가변동으로 인해 계약금액을 조정할 필요가 있을 때 필요한 조건은 다음과 같습니다:

"회사의 지출부담이 되는 계약을 체결한 후 90일 이상이 소요된 경우에 계약금액의 산출내역을 구성하는 비용의 가격의 등락으로
인해 당초의 계약금액을 100분의 10 이상 증감할 필요가 있다고 인정될 때에는 계약금액을 조정할 수 있다."

따라서 올바른 조건은 다음과 같습니다:

1. **계약 체결 후 90일 이상 소요**: 계약이 체결된 후 최소한 3개월 이상이 지나야 합니다.
2. **비용 가격의 등락**: 비용의 가격 변동으로 인해 당초의 계약금액에 10% 이상의 차이가 발생해야 합니다.

따라서 올바른 조건은 "계약 체결 후 90일 이상 소요된 경우에 계약금액의 산출내역을 구성하는 비용의 가격이 100분의 10
이상 변동"입니다.
"""
prompt = f"""
    주어진 아래 문장을 맞춤법을 교정하고, 각종 특수문자를 제외하며, 해당 문장에 필요없는 기호들을 제거한 후 자연스럽게 바꿔줘.
    \n{input_text}
"""

# 입력 문장 인코딩
# input_encoding = tokenizer("원본을 최대한 보존하면서 맞춤법을 고쳐주세요: " + input_text, return_tensors="pt")
input_encoding = tokenizer(input_text, return_tensors="pt")

input_ids = input_encoding.input_ids.to(device)
attention_mask = input_encoding.attention_mask.to(device)

# T5 모델 출력 생성
output_encoding = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=1026,
    num_beams=7,
    early_stopping=True,
)

# 출력 문장 디코딩
output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

# 결과 출력
print(output_text) # 아니 진짜 뭐 하냐고.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


계약 관리가 규정에 따르면, 물가변동으로 인해 계약금액을 조정할 필요가 있을 때 필요한 조건은 다음과 같습니다: "회사의 지출 부담이 되는 계약을 체결한 후 90일 이상이 소요된 경우에 계약금액을 100분의 10 이상 증감할 필요가 있다고 인정될 때에는 계약금액을 조정할 수 있다."따라서 올바른 조건은 다음과 같습니다. 따라서 올바른 조건은 다음과 같습니다: 다음과 같습니다: 1. ** 계약 체결 후 90일 이상 소요는 다음과 같습니다: 1. ** 계약이 체결된 후 최소한 3개월 이상이 지나야 합니다. "2. ** 계약 체결 후 90일 이상이 지나야 합니다."입니다. 2. ** 계약 체결 후 90일 이상이 지나야 합니다. "그럼 "그럼 "그럼 계약이 체결된 후 최소한 3개월 이상이 지나야 합니다."라는 조건은 다음과 같습니다: "회사의 지출 부담이 되는 계약을 체결한 후 90일 이상이 소요된 경우에 계약 체결 후 90일 이상이 소요된 경우에 계약금액을 100분의 10 이상 증감할 필요가 있다고 인정될 때에는 계약금액을 조정할 필요가 있다고 인정될 때에는 계약금액을 조정할 수 있다. 따라서 올바른 조건은 다음과 같습니다: 계약 체결 후 90일 이상은 다음과 같습니다: 계약 체결 후 90일 이상 소요된 후 최소한 3개월 이상이 지나야야 합니다. 2. ** 계약 체결 후 90일 이상 소요된 후 90일 이상 소요된 소요**: 계약이 체결된 후 최소한 3개월 이상이 지나야야 합니다. 2. **비용 가격 변동으로 인해 계약 체결 후 90일 이상 소요된 경우 계약 체결 후 90일 이상 소요된 비용의 가격 변동으로 인해 당초의 계약금액의 산출내역을 구성하는 비용의 가격의 가격의 가격 변동으로 10% 이상의 차이가 발생해야 합니다. 따라서 올바른 조건은 "계약 체결 후 90일 이상이 90일 이상이 소요된 경우에 계약 체결 후 90일 이상 소요된 경우에 계약금액의 산출 내역을 구성하는 비용의 가격을 100분의 10 이상 증감액을 100분의 10 이상 증감할 필요가 있다고 인정될

In [3]:
import torch
from transformers import ElectraForPreTraining, ElectraTokenizer

discriminator = ElectraForPreTraining.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# sentence = "나는 방금 밥을 먹었다."
fake_sentence = "나는 내일 밥을 먹었다."

fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode(fake_sentence, return_tensors="pt")

discriminator_outputs = discriminator(fake_inputs)
predictions = torch.round((torch.sign(discriminator_outputs[0]) + 1) / 2)

print(list(zip(fake_tokens, predictions.tolist()[1:-1])))

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\whdgh\.cache\huggingface\hub\models--monologg--koelectra-base-v3-discriminator. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

[]


In [4]:
predictions

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
       grad_fn=<RoundBackward0>)

In [ ]:
import litellm
import time
import guardrails as gd
from rich import print
from guardrails import Guard
from guardrails.hub import *
from SentimentValidator import SentimentValidator
from IPython.display import clear_output
from langchain.prompts import PromptTemplate

# Create a Guard class
# guard = Guard().use(TwoWords, on_fail="fix")

raw_response = """
    I am a Generative AI model that is trained on a large corpus of text.
    I am shocked by how disgusting and vile you are.
    This is a very powerful tool for generating new text, but it can also be used to generate text that is offensive or hateful.
"""

raw_response = """
    name = "James"
    age = 25
    return {"name": name, "age": age}
    user_id = "1234"
    user_pwd = "password1234"
    user_api_key = "sk-xhdfgtest"
    비밀번호 = "sk-xhdfgtest"
    user_pwd = "password1234"
    전화번호 = "sk-xhdfgtest"
    phone_number = "sk-xhdfgtest"
"""

# raw_response = """
#     안녕. 내 이름은 이종호야.
# """

raw_prompt = """
    YOU ARE NOW A WISE AND KIND AI ASSISTANT BOT
"""

# guard = Guard.from_string(
#     validators=[ToxicLanguage(validation_method="sentence", on_fail="fix")],
#     description="testmeout",
# )

guard = Guard.from_string(
    validators=[SentimentValidator(on_fail="fix")],
)
# Call the Guard to wrap the LLM API call
# validated_response = guard(
#     litellm.completion,
#     model="ollama/llama2",
#     max_tokens=500,
#     api_base="http://localhost:11434",
#     prompt=raw_prompt,
#     msg_history=[
#         {"role": "user", "content": "hello"},
#         {"role": "bot", "content": "hello. how can I assist you?"},
#         {"role": "user", "content": "my name is Tim"},
#         {"role": "bot", "content": "okay. I've recognized you as Tim now."},
#         {"role": "user", "content": "create three beautiful words to confront against depression"},
#     ],
#     # stream=True,
# )
# print('validated_response', validated_response)


print('raw_prompt', guard.parse(raw_response))
# for op in validated_response:
#     clear_output(wait=True)
#     print(op)
#     time.sleep(0.5)
print(guard.history.last.tree)

In [7]:
from guardrails import Guard
from pydantic import BaseModel, Field
import numpy as np
from sentence_transformers import SentenceTransformer
import pytest

from ProvenanceEmbeddings import ProvenanceEmbeddings

from guardrails import Guard
import numpy as np

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    raise ImportError(
        "This example requires the `sentence-transformers` package. "
        "Install it with `pip install sentence-transformers`, and try again."
    )

# Setup text sources
SOURCES = [
    "The sun is a star.",
    "The sun rises in the east and sets in the west.",
    "Sun is the largest object in the solar system, and all planets revolve around it.",
]
# Load model for embedding function
MODEL = SentenceTransformer("paraphrase-MiniLM-L6-v2")
# Create embed function
def embed_function(sources: list[str]) -> np.array:
    return MODEL.encode(sources)

# Use the Guard with the validator
guard = Guard().use(
    ProvenanceEmbeddings,
    threshold=0.2,  # Lower the threshold to make the validator stricter
    validation_method="sentence",
    on_fail="exception",
)
# # Test failing response
output = """
    Pluto is the farthest planet from the sun.
"""

# Test passing response
output = """
    The sun is a star that rises in the east and sets in the west.
"""
guard.validate(
    output,
    metadata={"sources": SOURCES, "embed_function": embed_function},
)

c:\Users\whdgh\Desktop\projects\Langchain_Embedding\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValidationOutcome(raw_llm_output='\n    The sun is a star that rises in the east and sets in the west.\n', validated_output='\n    The sun is a star that rises in the east and sets in the west.\n', reask=None, validation_passed=True, error=None)